# Bilinear Map
Bilinear Map is a very basic tool in state-of-the-art cryptography.

Let $  G\_1$, $G\_2$,$G\_T $ be multiplicative cyclic groups of prime order p.
And \\(g\_1\\),\\(g\_2\\) are generators of \\(G\_1\\), \\(G\_2\\). Then Bilinear Map
is a map \\(e: G\_1 \cdot G\_2 \rightarrow G\_T \\) such that for all \\(u \in G\_1\\),
\\(v \in G\_2\\) and \\(a,b \in Z_p\\), \\(e(u^a, v^b) = e(u, v)^{ab}\\).

## PBC-based demo
### PBC- Pairing-Based Cryptography Library
The [PBC(Pairing-Based Cryptography) library](https://crypto.stanford.edu/pbc/)
is a free C library built on [GMP(GNU Multiple Precision) library](https://gmplib.org/)
that performs the mathematical operations underlying pairing-based cryptosystems.

Speed and portability are the first concerns of PBC. Elliptic curves and much of number
theory are not prerequisite.

### Demo
The snippet below is a simple demo to verify the basic property of bilinear map :
\\( e(u^a, v^b) = e(u, v)^{ab} \\).

```c
#include <stdio.h> // for FILE
#include <pbc.h>
//no need to include gmp.h, pbc.h has already included.
int main()
{
	char s[16384];
	// read parameters from param_file as a array in s.
    // use the built-in a1.param parameters file.
	FILE *fp = fopen("xxx/pbc-0.5.14/param/a1.param","r");
	if(!fp) pbc_die("file opening error");

	size_t count = fread(s,1,16384,fp);
	if(!count) pbc_die("input error");

	// declare and initialize the pairing
	pairing_t pairing;
	pairing_init_set_str(pairing, s);

	// declare several element_t variables
	element_t x, y;

	// initialize all the element_t variables under pairing
	element_init_G1(x, pairing);  // x is some element of G1
	element_init_G2(y, pairing);  // y is some element of G2

	// randomly generate elements
	element_random(x);
	element_random(y);

	// applying the pairings
	pairing_pp_t pp;  // declare a paring with the same element

	/**
	 *  declare several mpz_t variables as exponents,
	 *  mpz_t are declared and implemented in GMP library.
	 */
	mpz_t a, b, ab;
	mpz_init_set_ui(a, 5);
	mpz_init_set_ui(b, 7);
	mpz_init_set_ui(ab, 35);

	element_t x_a, y_b;
	element_init_G1(x_a, pairing); // init x^a
	element_init_G2(y_b, pairing); // init y^b

	// exponentiate elements
	element_pp_t g_pp;
	element_pp_init(g_pp, x);     // set the base x
	element_pp_pow(x_a, a, g_pp); // cal x^a
	element_pp_init(g_pp, y);     // set the base y
	element_pp_pow(y_b, b, g_pp); // cal y^b

	// declare and initialize 2 elements in GT
	element_t ans1, ans2;
	element_init_GT(ans1, pairing);
	element_init_GT(ans2, pairing);

	// apply the pairings
	pairing_pp_init(pp, x_a, pairing);
	// cal e(x^a, y^b), result is stored in ans1
	pairing_pp_apply(ans1, y_b, pp);

	pairing_pp_init(pp, x, pairing);
	pairing_pp_apply(ans2, y, pp); // cal e(x, y)

	// exponentiate ans2
	element_pp_init(g_pp, ans2);
	element_pp_pow(ans2, ab, g_pp);

	// compare e(u^a, v^b) and e(u, v)^{ab}
	if(element_cmp(ans1, ans2) == 0)
		printf("e(u^a, v^b) equals to e(u, v)^{ab}, validation pass.\n");
	else printf("Not equal, validation fails.\n");

	// output the element
	/*
	element_printf("ans1: %B\n", ans1);
	element_printf("ans2: %B\n", ans2);
	*/
	return 0;
}

```